# Imports

In [13]:
import os
import pathlib

import torchvision

# Load data

In [14]:
data_dir = pathlib.Path.cwd().parent/'data'/'celebs'
os.makedirs(data_dir, exist_ok = True)

In [15]:
# # Download from PyTorch. Doesn't work because it is hosted on Google Drive server and it reached daily limit. Downloaded manually from another source.
# celeb_data = torchvision.datasets.CelebA(data_dir, split = 'all', download = True)


RuntimeError: The daily quota of the file img_align_celeba.zip is exceeded and it can't be downloaded. This is a limitation of Google Drive and can only be overcome by trying again later.